### **英雄特征处理**

该部分位于文件`feature/feature_process/hero_process.py`，为基本的内容信息

1. 类定义

初始化时创建了如下的信息：
- 归一化器
- 主阵营标识`self.main_camp`
- 己方英雄字典`self.main_camp_hero_dict`
- 敌方英雄字典`self.enemy_camp_hero_dict`
- 镜像阵营标识`self.transform_camp2_to_camp1`
- 读取特征配置`self.get_hero_config()` & 特征->归一化函数映射`self.map_feature_to_norm = self.normalizer.parse_config(self.hero_feature_config)`，对`.ini`文件进行解析
- 视距参数
- 单个英雄的特征维度
- 最多考虑的英雄数量

2. `get_hero_config`函数

从`hero_feature_config.ini`中获取相应的特征信息，存储在`self.feature_func_map`中

3. `process_vec_hero`函数

生成己方英雄特征并返回，综合调用了下面两个函数

4. `generate_hero_info_list`和`generate_hero_info_dict`

- 前者重要，后者会被前者覆盖
- 当`hero`符合当前阵营的情况下，`main_camp_hero_dict`存入`hero`信息，，否则存入敌方信息

5. `generate_one_type_hero_feature`函数

返回需要的`vector_feature`，此时所有的特征全部已经归一化

6. 剩下的工具函数，英雄血量，位置信息等




### **加入特征维度(1)**

根据数据协议可以加入如下的特征结构

```python
[feature_config]
# —— 基础状态（3）
is_hero_alive         = one_hot:1:eq
location_x            = min_max:-60000:60000
location_z            = min_max:-60000:60000

# —— 朝向（单位向量，2）
forward_x             = min_max:-1:1
forward_z             = min_max:-1:1

# —— 比例/标量（6）
hp_rate               = min_max:0:1
ep_rate               = min_max:0:1
level                 = min_max:1:15
money                 = min_max:0:20000
attack_range          = min_max:0:16000
is_in_grass           = one_hot:1:eq

# —— 核心属性（ActorValue，14）
phy_atk               = min_max:0:10000
phy_def               = min_max:0:10000
mgc_atk               = min_max:0:10000
mgc_def               = min_max:0:10000
mov_spd               = min_max:0:1200
atk_spd               = min_max:0:3000
crit_rate             = min_max:0:10000
crit_effe             = min_max:0:20000
phy_armor_hurt        = min_max:0:2000
mgc_armor_hurt        = min_max:0:2000
phy_vamp              = min_max:0:10000
mgc_vamp              = min_max:0:10000
cd_reduce             = min_max:0:4000
ctrl_reduce           = min_max:0:10000

# —— 技能槽（前4个，CD比例 + 可用性，8）
skill0_cd_rate        = min_max:0:1
skill1_cd_rate        = min_max:0:1
skill2_cd_rate        = min_max:0:1
skill3_cd_rate        = min_max:0:1
skill0_usable         = one_hot:1:eq
skill1_usable         = one_hot:1:eq
skill2_usable         = one_hot:1:eq
skill3_usable         = one_hot:1:eq

# —— K/D（2）
kill_cnt              = min_max:0:20
dead_cnt              = min_max:0:20

[feature_functions]
# —— 基础状态
is_hero_alive         = is_alive
location_x            = get_location_x
location_z            = get_location_z

# —— 朝向
forward_x             = get_forward_x
forward_z             = get_forward_z

# —— 比例/标量
hp_rate               = get_hp_rate
ep_rate               = get_ep_rate
level                 = get_level
money                 = get_money
attack_range          = get_attack_range
is_in_grass           = get_is_in_grass

# —— 核心属性（ActorValue）
phy_atk               = get_phy_atk
phy_def               = get_phy_def
mgc_atk               = get_mgc_atk
mgc_def               = get_mgc_def
mov_spd               = get_mov_spd
atk_spd               = get_atk_spd
crit_rate             = get_crit_rate
crit_effe             = get_crit_effe
phy_armor_hurt        = get_phy_armor_hurt
mgc_armor_hurt        = get_mgc_armor_hurt
phy_vamp              = get_phy_vamp
mgc_vamp              = get_mgc_vamp
cd_reduce             = get_cd_reduce
ctrl_reduce           = get_ctrl_reduce

# —— 技能槽
skill0_cd_rate        = get_skill_cd_rate
skill1_cd_rate        = get_skill_cd_rate
skill2_cd_rate        = get_skill_cd_rate
skill3_cd_rate        = get_skill_cd_rate
skill0_usable         = get_skill_usable
skill1_usable         = get_skill_usable
skill2_usable         = get_skill_usable
skill3_usable         = get_skill_usable

# —— K/D
kill_cnt              = get_kill_cnt
dead_cnt              = get_dead_cnt

```

同时在`hero_process.py`中加入特征处理如下：

```python
    def get_forward_x(self, hero, out, feature_name):
        fx = hero.get("actor_state", {}).get("forward", {}).get("x", 0.0)
        fz = hero.get("actor_state", {}).get("forward", {}).get("z", 0.0)
        denom = math.sqrt(fx * fx + fz * fz) + 1e-8
        val = fx / denom
        if self.transform_camp2_to_camp1:
            val = -val
        out.append(float(val))  # ∈[-1,1]，INI 用 min_max:-1:1

    def get_forward_z(self, hero, out, feature_name):
        fx = hero.get("actor_state", {}).get("forward", {}).get("x", 0.0)
        fz = hero.get("actor_state", {}).get("forward", {}).get("z", 0.0)
        denom = math.sqrt(fx * fx + fz * fz) + 1e-8
        val = fz / denom
        if self.transform_camp2_to_camp1:
            val = -val
        out.append(float(val))  # ∈[-1,1]

    # ========== 比例/标量 ==========
    def get_hp_rate(self, hero, out, feature_name):
        st = hero.get("actor_state", {})
        hp = float(st.get("hp", 0.0))
        max_hp = float(st.get("max_hp", 1.0))
        out.append(0.0 if max_hp <= 0 else hp / max_hp)

    def get_ep_rate(self, hero, out, feature_name):
        vals = hero.get("actor_state", {}).get("values", {})
        ep = float(vals.get("ep", 0.0))
        max_ep = float(vals.get("max_ep", 1.0))
        out.append(0.0 if max_ep <= 0 else ep / max_ep)

    def get_level(self, hero, out, feature_name):
        out.append(float(hero.get("level", 0)))

    def get_money(self, hero, out, feature_name):
        out.append(float(hero.get("money", 0)))

    def get_attack_range(self, hero, out, feature_name):
        out.append(float(hero.get("actor_state", {}).get("attack_range", 0)))

    # ========== 核心属性（ActorValue） ==========
    def _get_val(self, hero, key, default=0.0):
        return float(hero.get("actor_state", {}).get("values", {}).get(key, default))

    def get_phy_atk(self, hero, out, feature_name): out.append(self._get_val(hero, "phy_atk"))
    def get_phy_def(self, hero, out, feature_name): out.append(self._get_val(hero, "phy_def"))
    def get_mgc_atk(self, hero, out, feature_name): out.append(self._get_val(hero, "mgc_atk"))
    def get_mgc_def(self, hero, out, feature_name): out.append(self._get_val(hero, "mgc_def"))
    def get_mov_spd(self, hero, out, feature_name): out.append(self._get_val(hero, "mov_spd"))
    def get_atk_spd(self, hero, out, feature_name): out.append(self._get_val(hero, "atk_spd"))
    def get_crit_rate(self, hero, out, feature_name): out.append(self._get_val(hero, "crit_rate"))
    def get_crit_effe(self, hero, out, feature_name): out.append(self._get_val(hero, "crit_effe"))
    def get_phy_armor_hurt(self, hero, out, feature_name): out.append(self._get_val(hero, "phy_armor_hurt"))
    def get_mgc_armor_hurt(self, hero, out, feature_name): out.append(self._get_val(hero, "mgc_armor_hurt"))
    def get_phy_vamp(self, hero, out, feature_name): out.append(self._get_val(hero, "phy_vamp"))
    def get_mgc_vamp(self, hero, out, feature_name): out.append(self._get_val(hero, "mgc_vamp"))
    def get_cd_reduce(self, hero, out, feature_name): out.append(self._get_val(hero, "cd_reduce"))
    def get_ctrl_reduce(self, hero, out, feature_name): out.append(self._get_val(hero, "ctrl_reduce"))

    # ========== 技能：前 4 个槽（含召唤师技能） ==========
    def _get_slot(self, hero, idx):
        slots = hero.get("skill_state", {}).get("slot_states", [])
        if 0 <= idx < len(slots):
            return slots[idx]
        return None

    def get_skill_cd_rate(self, hero, out, feature_name):
        # 特征名形如：skill0_cd_rate / skill1_cd_rate / skill2_cd_rate / skill3_cd_rate
        # 从特征名尾部解析索引
        idx = int(''.join([c for c in feature_name if c.isdigit()]) or 0)
        slot = self._get_slot(hero, idx)
        if not slot:
            out.append(0.0); return
        cd = float(slot.get("cooldown", 0.0))
        cd_max = float(slot.get("cooldown_max", 0.0))
        rate = 0.0 if cd_max <= 0 else (cd / cd_max)
        out.append(rate)  # 0~1

    def get_skill_usable(self, hero, out, feature_name):
        idx = int(''.join([c for c in feature_name if c.isdigit()]) or 0)
        slot = self._get_slot(hero, idx)
        usable = slot.get("usable", False) if slot else False
        out.append(1.0 if usable else 0.0)

    # ========== 其它语义 ==========
    def get_is_in_grass(self, hero, out, feature_name):
        out.append(1.0 if hero.get("isInGrass", False) else 0.0)

    def get_kill_cnt(self, hero, out, feature_name):
        out.append(float(hero.get("killCnt", 0)))

    def get_dead_cnt(self, hero, out, feature_name):
        out.append(float(hero.get("deadCnt", 0)))
```

需要注意的是在`conf.py`文件中需要将对应的英雄特征维度修改为当前的70维(35*2)